In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
os.chdir("/mnt/lareaulab/reliscu/projects/Chronocell/data")

In [3]:
legend = pd.read_csv("experimental_data_legend.csv")
legend.head()

,File,Type,Species,Cell_type,Method,Experiment_type,Quantity_description,Calculation_method,Notes
0,Eraslan2019_HPA_tableS3.csv,TE,Human,29 tissues,"LC‐MS/MS, RNA-seq",In vivo,Protein‐to‐mRNA ratio,Log-10-transformed ratio of the normalized pro...,NaN
1,Hernandez-Alias2023_PTR_GTEx_tableS2.csv,TE,Human,24 tissues,"TMT MS3, RNA-seq",In vivo,Protein‐to‐mRNA ratio,Log-10-transformed ratio of the normalized pro...,NaN
2,Zheng2025_human_tableS1.csv,TE,Human,>140 cell types,"Ribo-seq, RNA-seq",In vivo; In vitro,Translation efficiency,NaN,"Based on 3,819 ribosomal profiling datasets"
3,Zheng2025_mouse_tableS1.csv,TE,Mouse,>140 cell types,"Ribo-seq, RNA-seq",In vivo; In vitro,Translation efficiency,NaN,"Based on 3,819 ribosomal profiling datasets"
4,Hernandez-Alias2023_TE_Wang2020_tableS2.csv,TE,Human,Brain,"Ribo-seq, RNA-seq",In vivo,Translation efficiency,Median of log-10-transformed Ribo-seq TPM and ...,NaN


In [4]:
mask = legend['Type'].values == "TE"
legend['Path'] = None
legend.loc[mask, 'Path'] = (os.getcwd() + "/Translation_rates/" + legend['File'].values)[mask]
legend.loc[~mask, 'Path'] = (os.getcwd() + "/Protein_degradation_rates/" + legend['File'].values)[~mask]

## Start with human half-life datasets

In [5]:
mask = (legend['Type'] == "Deg") & (legend['Species'] == "Human")
legend1 = legend.loc[mask].reset_index(drop=True)

In [52]:
idx = 0
data0 = pd.read_csv(legend1.loc[idx, 'Path'])
data0.head()

,Protein Accession,Gene,Description,Whole Turnover,Whole Half-Life,Whole Chi2,Cytoplasm Turnover,Cytoplasm Half-Life,Cytoplasm Chi2,Nucleus Turnover,Nucleus Half-Life,Nucleus Chi2,Nucleolus Turnover,Nucleolus Half-Life,Nucleolus Chi2
0,IPI00788723,NaN,similar to hCG1777996,26.045200,NaN,0.006633,NaN,NaN,NaN,NaN,NaN,NaN,24.034901,NaN,NaN
1,IPI00292537,NaN,Isoform 2 of Nuclear factor NF-kappa-B p105 su...,23.990700,NaN,0.043440,0.000000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN,0.000000
2,IPI00018783,ITPA,Inosine triphosphate pyrophosphatase,22.706051,NaN,0.014058,22.577850,-598.668696,0.012268,25.80835,999.000000,0.038288,NaN,NaN,NaN
3,IPI00059279,EXOC4,Exocyst complex component 4,20.688351,NaN,0.006546,16.347550,999.000000,0.016951,24.11875,157.231155,0.000501,13.062000,NaN,NaN
4,IPI00410079,FAM82A2,Isoform 1 of Regulator of microtubule dynamics...,24.819050,NaN,0.008621,23.186899,999.000000,0.013351,24.17310,50.688920,0.000456,27.525101,999.0,0.016516


In [53]:
# Replace negative values with NaN (indicates poor fit model fit)
data0.loc[data0['Cytoplasm Half-Life'] < 0, 'Cytoplasm Half-Life'] = np.nan

# Standardize columns
data0['Half-life'] = data0['Cytoplasm Half-Life']
data0['Cell_type'] = legend1.loc[idx, 'Cell_type']
data0['Method'] = legend1.loc[idx, 'Method']

data0.head()

,Protein Accession,Gene,Description,Whole Turnover,Whole Half-Life,Whole Chi2,Cytoplasm Turnover,Cytoplasm Half-Life,Cytoplasm Chi2,Nucleus Turnover,Nucleus Half-Life,Nucleus Chi2,Nucleolus Turnover,Nucleolus Half-Life,Nucleolus Chi2,Half-life,Cell_type,Method
0,IPI00788723,NaN,similar to hCG1777996,26.045200,NaN,0.006633,NaN,NaN,NaN,NaN,NaN,NaN,24.034901,NaN,NaN,NaN,HeLa cells,SILAC pulse-chase
1,IPI00292537,NaN,Isoform 2 of Nuclear factor NF-kappa-B p105 su...,23.990700,NaN,0.043440,0.000000,NaN,0.000000,0.00000,NaN,0.000000,0.000000,NaN,0.000000,NaN,HeLa cells,SILAC pulse-chase
2,IPI00018783,ITPA,Inosine triphosphate pyrophosphatase,22.706051,NaN,0.014058,22.577850,NaN,0.012268,25.80835,999.000000,0.038288,NaN,NaN,NaN,NaN,HeLa cells,SILAC pulse-chase
3,IPI00059279,EXOC4,Exocyst complex component 4,20.688351,NaN,0.006546,16.347550,999.0,0.016951,24.11875,157.231155,0.000501,13.062000,NaN,NaN,999.0,HeLa cells,SILAC pulse-chase
4,IPI00410079,FAM82A2,Isoform 1 of Regulator of microtubule dynamics...,24.819050,NaN,0.008621,23.186899,999.0,0.013351,24.17310,50.688920,0.000456,27.525101,999.0,0.016516,999.0,HeLa cells,SILAC pulse-chase


In [54]:
idx = 1
data1 = pd.read_csv(legend1.loc[idx, 'Path'])
data1.head()

,Protein IDs,Gene Names,Protein Descriptions,Uniprot,Proteins,Peptides (seq),Unique Peptides (seq),Mol. Weight [kDa],Ratio H/L,Ratio H/L Count,Intensity,half-life in h,kdeg,RIA
0,IPI00298409,PDCL,Phosducin-like protein,Q13371;Q4VXB6;Q9UNX1,1,2,2,34.28,14.65,4,5860100,8.1,0.08572,0.4361
1,IPI00299214;IPI00791117;IPI00794651;IPI00795236,TK1,"thymidine kinase 1, soluble;29 kDa protein;20 ...",P04183,4,3,3,29.89,12.76,5,9627000,8.6,0.08034,0.4273
2,IPI00179328;IPI00549924;IPI00000163;IPI00643624,CTNNAL1;RP11-115J22.1-005,Isoform 1 of Alpha-catulin;Isoform 2 of Alpha-...,Q9UBT7-1;Q9UBT7;Q9UBT7-2;Q9UBT7-3;Q5JTQ9;Q5JTQ6,4,4,4,81.90,12.40,7,12889000,8.7,0.07926,0.4254
3,IPI00847321;IPI00022748,TEGT;BI1;TMBIM6,testis enhanced gene transcript (BAX inhibitor...,A8MUR1;P55061,2,2,2,32.71,12.12,3,4025200,8.8,0.07837,0.4238
4,IPI00013241,UBL5,Ubiquitin-like protein 5,Q9BZL1,1,2,2,8.55,11.88,3,892020,8.9,0.07761,0.4224


In [ ]:
# Separate genes with multiple entries
data1['Gene Names'] = data1['Gene Names'].str.split(';')
data1 = data1.explode('Gene Names')
data1['Gene Names'] = data1['Gene Names'].str.strip()

# Standardize column names
data1['Half-life'] = data1['half-life in h']
data1['Cell_type'] = legend1.loc[idx, 'Cell_type']
data1['Method'] = legend1.loc[idx, 'Method']

data1.head()

,Protein IDs,Gene Names,Protein Descriptions,Uniprot,Proteins,Peptides (seq),Unique Peptides (seq),Mol. Weight [kDa],Ratio H/L,Ratio H/L Count,Intensity,half-life in h,kdeg,RIA,Half-life,Cell_type,Method
0,IPI00298409,PDCL,Phosducin-like protein,Q13371;Q4VXB6;Q9UNX1,1,2,2,34.28,14.65,4,5860100,8.1,0.08572,0.4361,8.1,HeLa cells,SILAC pulse-chase
1,IPI00299214;IPI00791117;IPI00794651;IPI00795236,TK1,"thymidine kinase 1, soluble;29 kDa protein;20 ...",P04183,4,3,3,29.89,12.76,5,9627000,8.6,0.08034,0.4273,8.6,HeLa cells,SILAC pulse-chase
2,IPI00179328;IPI00549924;IPI00000163;IPI00643624,CTNNAL1,Isoform 1 of Alpha-catulin;Isoform 2 of Alpha-...,Q9UBT7-1;Q9UBT7;Q9UBT7-2;Q9UBT7-3;Q5JTQ9;Q5JTQ6,4,4,4,81.90,12.40,7,12889000,8.7,0.07926,0.4254,8.7,HeLa cells,SILAC pulse-chase
2,IPI00179328;IPI00549924;IPI00000163;IPI00643624,RP11-115J22.1-005,Isoform 1 of Alpha-catulin;Isoform 2 of Alpha-...,Q9UBT7-1;Q9UBT7;Q9UBT7-2;Q9UBT7-3;Q5JTQ9;Q5JTQ6,4,4,4,81.90,12.40,7,12889000,8.7,0.07926,0.4254,8.7,HeLa cells,SILAC pulse-chase
3,IPI00847321;IPI00022748,TEGT,testis enhanced gene transcript (BAX inhibitor...,A8MUR1;P55061,2,2,2,32.71,12.12,3,4025200,8.8,0.07837,0.4238,8.8,HeLa cells,SILAC pulse-chase


In [13]:
idx = 2
data2 = pd.read_csv(legend1.loc[idx, 'Path'])
data2.head()

,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_6,CHX_6.1,CHX_8,CHX_8.1,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived
0,Q9Y3B1,PRELID3B,0.96,0.99,1.00,1.05,0.14,0.20,0.09,0.06,...,0.09,0.07,0.11,0.09,0.09,0.10,non-linear model,0.33,1.00,short-lived
1,Q14494,NFE2L1,0.90,1.03,1.02,1.04,0.17,0.19,0.12,0.16,...,0.04,0.07,0.08,0.08,0.08,0.08,non-linear model,0.37,0.99,short-lived
2,Q13794,PMAIP1,0.93,1.00,1.02,1.05,0.31,0.34,0.24,0.24,...,0.28,0.27,0.33,0.29,0.34,0.30,non-linear model,0.43,0.98,short-lived
3,P01106-2,MYC,0.92,0.90,1.06,1.11,0.22,0.24,0.17,0.18,...,0.09,0.08,0.10,0.10,0.09,0.10,non-linear model,0.44,0.98,short-lived
4,Q15828,CST6,1.07,0.94,1.04,0.95,0.25,0.25,0.07,0.11,...,0.11,0.10,0.12,0.11,0.11,0.10,non-linear model,0.44,0.99,short-lived


In [14]:
# Standardize column names
data2['Gene'] = data2['Gene symbol']
data2['Cell_type'] = legend1.loc[idx, 'Cell_type']
data2['Method'] = legend1.loc[idx, 'Method']

data2.head()

,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.1,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method
0,Q9Y3B1,PRELID3B,0.96,0.99,1.00,1.05,0.14,0.20,0.09,0.06,...,0.09,0.09,0.10,non-linear model,0.33,1.00,short-lived,PRELID3B,HCT116 colon cancer-derived epithelial cells,CHX chase
1,Q14494,NFE2L1,0.90,1.03,1.02,1.04,0.17,0.19,0.12,0.16,...,0.08,0.08,0.08,non-linear model,0.37,0.99,short-lived,NFE2L1,HCT116 colon cancer-derived epithelial cells,CHX chase
2,Q13794,PMAIP1,0.93,1.00,1.02,1.05,0.31,0.34,0.24,0.24,...,0.29,0.34,0.30,non-linear model,0.43,0.98,short-lived,PMAIP1,HCT116 colon cancer-derived epithelial cells,CHX chase
3,P01106-2,MYC,0.92,0.90,1.06,1.11,0.22,0.24,0.17,0.18,...,0.10,0.09,0.10,non-linear model,0.44,0.98,short-lived,MYC,HCT116 colon cancer-derived epithelial cells,CHX chase
4,Q15828,CST6,1.07,0.94,1.04,0.95,0.25,0.25,0.07,0.11,...,0.11,0.11,0.10,non-linear model,0.44,0.99,short-lived,CST6,HCT116 colon cancer-derived epithelial cells,CHX chase


In [15]:
idx = 3
data3 = pd.read_csv(legend1.loc[idx, 'Path'])

# Standardize column names
data3['Gene'] = data3['Gene symbol']
data3['Cell_type'] = legend1.loc[idx, 'Cell_type']
data3['Method'] = legend1.loc[idx, 'Method']
data3.head()

,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.1,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method
0,P30304,CDC25A,0.96,0.98,0.91,1.15,0.25,0.22,0.26,0.25,...,0.21,0.22,0.23,non-linear model,0.20,0.98,short-lived,CDC25A,HEK293T kidney cells,CHX chase
1,Q9Y3B1,PRELID3B,0.98,0.97,0.83,1.22,0.13,0.19,0.10,0.07,...,0.11,0.08,0.07,non-linear model,0.30,0.96,short-lived,PRELID3B,HEK293T kidney cells,CHX chase
2,P51959,CCNG1,0.98,0.97,1.01,1.04,0.33,0.28,0.29,0.29,...,0.23,0.23,0.24,non-linear model,0.44,0.99,short-lived,CCNG1,HEK293T kidney cells,CHX chase
3,Q8IYN2,TCEAL8,0.98,1.04,1.03,0.95,0.23,0.23,0.10,0.08,...,0.05,0.04,0.05,non-linear model,0.46,1.00,short-lived,TCEAL8,HEK293T kidney cells,CHX chase
4,Q6PEV8,FAM199X,1.00,0.96,0.95,1.09,0.28,0.23,0.09,0.10,...,0.08,0.04,0.06,non-linear model,0.47,0.99,short-lived,FAM199X,HEK293T kidney cells,CHX chase


In [16]:
idx = 4
data4 = pd.read_csv(legend1.loc[idx, 'Path'])

# Standardize column names
data4['Gene'] = data4['Gene symbol']
data4['Cell_type'] = legend1.loc[idx, 'Cell_type']
data4['Method'] = legend1.loc[idx, 'Method']
data4.head()

,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.1,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method
0,P57086,SCAND1,1.18,0.86,0.99,0.97,0.17,0.28,0.20,0.12,...,0.18,0.21,0.19,non-linear model,0.34,0.97,short-lived,SCAND1,RPE-1 retinal pigmented epithelial cells,CHX chase
1,P24385,CCND1,1.39,0.73,0.53,1.35,0.28,0.16,0.11,0.10,...,0.08,0.08,0.07,non-linear model,0.42,0.81,short-lived,CCND1,RPE-1 retinal pigmented epithelial cells,CHX chase
2,P52823,STC1,1.02,0.86,1.15,0.97,0.31,0.37,0.26,0.23,...,0.37,0.46,0.26,non-linear model,0.44,0.94,short-lived,STC1,RPE-1 retinal pigmented epithelial cells,CHX chase
3,O76061,STC2,1.04,0.91,1.05,1.00,0.37,0.31,0.25,0.26,...,0.30,0.29,0.31,non-linear model,0.47,0.99,short-lived,STC2,RPE-1 retinal pigmented epithelial cells,CHX chase
4,Q8IYN2,TCEAL8,0.83,0.92,1.09,1.16,0.07,0.01,0.07,0.12,...,0.00,0.08,0.00,manual inspection,0.52,1.00,short-lived,TCEAL8,RPE-1 retinal pigmented epithelial cells,CHX chase


In [17]:
idx = 5
data5 = pd.read_csv(legend1.loc[idx, 'Path'])

# Standardize column names
data5['Gene'] = data5['Gene symbol']
data5['Cell_type'] = legend1.loc[idx, 'Cell_type']
data5['Method'] = legend1.loc[idx, 'Method']
data5.head()

,Uniprot ID,Gene symbol,CHX_0,CHX_0.1,CHX_0.2,CHX_0.3,CHX_1,CHX_1.1,CHX_2,CHX_2.1,...,CHX_8.1,CHX_8.2,CHX_8.3,Model,Half-life,R2,Short-lived,Gene,Cell_type,Method
0,Q9Y3B1,PRELID3B,1.08,0.96,0.98,0.98,0.30,0.19,0.13,0.24,...,0.24,0.21,0.15,non-linear model,0.34,0.99,short-lived,PRELID3B,U2OS tibia cells,CHX chase
1,Q96E09,FAM122A,1.09,0.93,0.96,1.02,0.28,0.24,0.11,0.16,...,0.13,0.18,0.10,non-linear model,0.43,0.99,short-lived,FAM122A,U2OS tibia cells,CHX chase
2,P30279,CCND2,1.06,0.98,1.02,0.93,0.27,0.17,0.11,0.08,...,0.06,0.08,0.08,non-linear model,0.43,0.99,short-lived,CCND2,U2OS tibia cells,CHX chase
3,Q9Y6B2,EID1,1.02,0.99,0.98,1.01,0.22,0.21,0.09,0.09,...,0.06,0.05,0.06,non-linear model,0.43,1.00,short-lived,EID1,U2OS tibia cells,CHX chase
4,Q8NA92,THAP8,0.91,0.92,1.13,1.04,0.30,0.36,0.29,0.31,...,0.27,0.34,0.34,non-linear model,0.46,0.96,short-lived,THAP8,U2OS tibia cells,CHX chase


In [38]:
idx = 6
data6 = pd.read_csv(legend1.loc[idx, 'Path'])
data6.head()

,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Hepatocytes replicate 1 R_sq,Hepatocytes replicate 2 half_life,Hepatocytes replicate 2 dataQual,Hepatocytes replicate 2 R_sq,Monocytes replicate 1 half_life,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.885,24.377,good,0.905,14.599,weak,0.978,NaN,NaN,NaN
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,NaN,145.343,weak,0.852,78.090,weak,0.995,57.957,weak,0.987
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,69.614,weak,0.950,52.862,weak,0.949
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,76.276,weak,0.982,59.539,good,0.988
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,68.136,weak,0.938,51.380,weak,0.985


In [39]:
# Average half-life values over replicates for B cells
pattern = r'^Bcells replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data6.filter(regex=pattern).columns
data6['Half-life'] = data6[matching_cols].mean(axis=1)

# Standardize column names
data6['Gene'] = data6['gene_name']
data6['Cell_type'] = legend1.loc[idx, 'Cell_type']
data6['Method'] = legend1.loc[idx, 'Method']
data6.head()

,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 half_life,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.599,weak,0.978,NaN,NaN,NaN,NaN,A2M,B cells,Dynamic SILAC
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,78.090,weak,0.995,57.957,weak,0.987,81.8375,AAAS,B cells,Dynamic SILAC
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,69.614,weak,0.950,52.862,weak,0.949,30.5240,AACS,B cells,Dynamic SILAC
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,76.276,weak,0.982,59.539,good,0.988,NaN,AAK1,B cells,Dynamic SILAC
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.136,weak,0.938,51.380,weak,0.985,NaN,AAMDC,B cells,Dynamic SILAC


In [40]:
data6[matching_cols]

,Bcells replicate 1 half_life,Bcells replicate 2 half_life
0,NaN,NaN
1,75.299,88.376
2,NaN,30.524
3,NaN,NaN
4,NaN,NaN
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [41]:
# NK cells

idx = 7
data7 = pd.read_csv(legend1.loc[idx, 'Path'])

# Average half-life values over replicates
pattern = r'^NK cells replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data7.filter(regex=pattern).columns
data7['Half-life'] = data7[matching_cols].mean(axis=1)

# Standardize column names
data7['Gene'] = data7['gene_name']
data7['Cell_type'] = legend1.loc[idx, 'Cell_type']
data7['Method'] = legend1.loc[idx, 'Method']
data7.head()

,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 half_life,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.599,weak,0.978,NaN,NaN,NaN,NaN,A2M,Monocytes,Dynamic SILAC
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,78.090,weak,0.995,57.957,weak,0.987,293.8175,AAAS,Monocytes,Dynamic SILAC
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,69.614,weak,0.950,52.862,weak,0.949,98.5740,AACS,Monocytes,Dynamic SILAC
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,76.276,weak,0.982,59.539,good,0.988,152.6780,AAK1,Monocytes,Dynamic SILAC
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.136,weak,0.938,51.380,weak,0.985,NaN,AAMDC,Monocytes,Dynamic SILAC


In [42]:
data7[matching_cols]

,NK cells replicate 1 half_life,NK cells replicate 2 half_life
0,NaN,NaN
1,379.453,208.182
2,NaN,98.574
3,NaN,152.678
4,NaN,NaN
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [35]:
# Monocytes

idx = 8
data8 = pd.read_csv(legend1.loc[idx, 'Path'])

# Average half-life values over replicates
pattern = r'^Monocytes replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data8.filter(regex=pattern).columns
data8['Half-life'] = data8[matching_cols].mean(axis=1)

# Standardize column names
data8['Gene'] = data8['gene_name']
data8['Cell_type'] = legend1.loc[idx, 'Cell_type']
data8['Method'] = legend1.loc[idx, 'Method']
data8.head()

,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 half_life,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.599,weak,0.978,NaN,NaN,NaN,14.5990,A2M,NK cells,Dynamic SILAC
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,78.090,weak,0.995,57.957,weak,0.987,68.0235,AAAS,NK cells,Dynamic SILAC
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,69.614,weak,0.950,52.862,weak,0.949,61.2380,AACS,NK cells,Dynamic SILAC
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,76.276,weak,0.982,59.539,good,0.988,67.9075,AAK1,NK cells,Dynamic SILAC
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.136,weak,0.938,51.380,weak,0.985,59.7580,AAMDC,NK cells,Dynamic SILAC


In [36]:
data8[matching_cols]

,Monocytes replicate 1 half_life,Monocytes replicate 2 half_life
0,14.599,NaN
1,78.090,57.957
2,69.614,52.862
3,76.276,59.539
4,68.136,51.380
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [32]:
# Hepatocytes

idx = 9
data9 = pd.read_csv(legend1.loc[idx, 'Path'])

# Average half-life values over replicates
pattern = r'^Hepatocytes replicate \d+ half_life'  # regex for exact match with digits at end
matching_cols = data9.filter(regex=pattern).columns
data9['Half-life'] = data9[matching_cols].mean(axis=1)

# Standardize column names
data9['Gene'] = data9['gene_name']
data9['Cell_type'] = legend1.loc[idx, 'Cell_type']
data9['Method'] = legend1.loc[idx, 'Method']
data9.head()

,gene_name,Bcells replicate 1 half_life,Bcells replicate 1 dataQual,Bcells replicate 1 R_sq,Bcells replicate 2 half_life,Bcells replicate 2 dataQual,Bcells replicate 2 R_sq,NK cells replicate 1 half_life,NK cells replicate 1 dataQual,NK cells replicate 1 R_sq,...,Monocytes replicate 1 half_life,Monocytes replicate 1 dataQual,Monocytes replicate 1 R_sq,Monocytes replicate 2 half_life,Monocytes replicate 2 dataQual,Monocytes replicate 2 R_sq,Half-life,Gene,Cell_type,Method
0,A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.599,weak,0.978,NaN,NaN,NaN,24.0255,A2M,Hepatocytes,Dynamic SILAC
1,AAAS,75.299,good,0.982,88.376,good,0.990,379.453,good,0.961,...,78.090,weak,0.995,57.957,weak,0.987,145.3430,AAAS,Hepatocytes,Dynamic SILAC
2,AACS,NaN,NaN,NaN,30.524,weak,0.995,NaN,NaN,NaN,...,69.614,weak,0.950,52.862,weak,0.949,NaN,AACS,Hepatocytes,Dynamic SILAC
3,AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,76.276,weak,0.982,59.539,good,0.988,NaN,AAK1,Hepatocytes,Dynamic SILAC
4,AAMDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.136,weak,0.938,51.380,weak,0.985,NaN,AAMDC,Hepatocytes,Dynamic SILAC


In [33]:
data9[matching_cols]

,Hepatocytes replicate 1 half_life,Hepatocytes replicate 2 half_life
0,23.674,24.377
1,NaN,145.343
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
8799,NaN,NaN
8800,NaN,NaN
8801,NaN,NaN
8802,NaN,NaN


In [56]:
idx = 10
data10 = pd.read_csv(legend1.loc[idx, 'Path'])
data10.head()

,gene_name,Sample,median_kdeg,median_Thalf,found_in_replicates
0,1110004f10rik|smap,mouse,0.020362,34.041064,3
1,1110004f10rik|smap,mouse_2DG,0.012545,55.251729,1
2,2210016l21rik|custos,mouse,0.040182,17.250261,3
3,2210016l21rik|custos,mouse_2DG,0.032057,21.622114,1
4,2310022a10rik,mouse,0.046491,14.909245,3


In [57]:
# Subset to human data
data10 = data10[data10['Sample'].str.contains('human')]

# Separate genes with multiple entries
data10['gene_name'] = data10['gene_name'].str.split(';')
data10 = data10.explode('gene_name')
data10['gene_name'] = data10['gene_name'].str.strip()

# Standardize columns
data10['Gene'] = data10['gene_name']
data10['Half-life'] = data10['median_Thalf']
data10['Cell_type'] = legend1.loc[idx, 'Cell_type']
data10['Method'] = legend1.loc[idx, 'Method']

data10.head()

,gene_name,Sample,median_kdeg,median_Thalf,found_in_replicates,Gene,Half-life,Cell_type,Method
12,A0A0B4J2A0|SHPK,human,0.009091,76.243850,3,A0A0B4J2A0|SHPK,76.243850,Induced presomitic mesoderm cells,Dynamic SILAC
13,A0A2R8YDQ0|VPS33B,human,0.019075,36.338478,2,A0A2R8YDQ0|VPS33B,36.338478,Induced presomitic mesoderm cells,Dynamic SILAC
14,A0A2R8YFF1|AGGF1,human,0.051287,13.515126,2,A0A2R8YFF1|AGGF1,13.515126,Induced presomitic mesoderm cells,Dynamic SILAC
15,A0A3B3ISV5|LRRC57,human,0.026352,26.303862,3,A0A3B3ISV5|LRRC57,26.303862,Induced presomitic mesoderm cells,Dynamic SILAC
16,A0A3B3ITX4|ATP5MG,human,0.002528,274.208368,2,A0A3B3ITX4|ATP5MG,274.208368,Induced presomitic mesoderm cells,Dynamic SILAC
